In [ ]:
# 2020-11-1 created by Akson

In [ ]:
# Code17.1
# import

import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Code17.2
# generate 3-d data function

def generate_3d_data(m, w1 = 0.1, w2 = 0.3, noise = 0.1):
    angles = np.random.rand(m) * 3 * np.pi / 2 - 0.5
    data = np.empty((m, 3))
    data[:, 0] = np.cos(angles) + np.sin(angles) / 2 + noise * np.random.randn(m) / 2
    data[:, 1] = np.sin(angles) * 0.7 + noise * np.random.randn(m) / 2
    data[:, 2] = data[:, 0] * w1 + data[:, 1] * w2 + noise * np.random.randn(m)

    return data

In [ ]:
# Code17.3
# create train dataset

X_train = generate_3d_data(60)
X_train = X_train - X_train.mean(axis = 0, keepdims = 0)

In [ ]:
# Code17.4
# create model and train

# simple encoder network
encoder = keras.models.Sequential([keras.layers.Dense(2, input_shape = [3])])
# simple decoder network
decoder = keras.models.Sequential([keras.layers.Dense(3, input_shape = [2])])
# group
autoencoder = keras.models.Sequential([encoder, decoder])

autoencoder.compile(loss = 'mse', optimizer = keras.optimizers.SGD(lr = 1.5))
# 用整个网络来拟合训练数据
history = autoencoder.fit(X_train, X_train, epochs = 20)

In [ ]:
# Code17.5
# only use encoder 

codings = encoder.predict(X_train)

In [ ]:
# Code17.6
# load fashion mnist for stack-autoencoder

(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()
# standard data
X_train_full = X_train_full.astype(np.float32) / 255
X_test = X_test.astype(np.float32) / 255
X_train = X_train_full[: -5000]
X_valid = X_train_full[-5000: ]
y_train = y_train_full[: -5000]
y_valid = y_train_full[-5000: ]

In [ ]:
# Code17.7
# create stack-autoencoder model

stacked_encoder = keras.models.Sequential([
    keras.layers.Flatten(input_shape = [28, 28]),
    keras.layers.Dense(100, activation = 'selu'),
    keras.layers.Dense(30, activation = 'selu')
])

stacked_decoder = keras.models.Sequential([
    keras.layers.Dense(100, activation = 'selu', input_shape = [30]),
    keras.layers.Dense(28 * 28, activation = 'sigmoid'),
    keras.layers.Reshape([28, 28])
])

stacked_autoencoder = keras.models.Sequential([stacked_encoder, stacked_decoder])

In [ ]:
# Code17.8
# train model

def rounded_accuracy(y_true, y_pred):
    return keras.metrics.binary_accuracy(tf.round(y_true), tf.round(y_pred))

stacked_autoencoder.compile(loss = 'binary_crossentropy', optimizer = keras.optimizers.SGD(lr = 1.5), metrics = [rounded_accuracy])
history = stacked_autoencoder.fit(X_train, X_train, epochs = 20, validation_data = (X_valid, X_valid))

In [ ]:
# Code17.9
# draw result

def plot_image(image):
    plt.imshow(image, cmap = 'binary')
    plt.axis('off')

def show_reconstructions(model, images = X_valid, n_images = 5):
    reconstructions = model.predict(images[:n_images])
    fig = plt.figure(figsize = (n_images * 1.5, 3))
    for image_index in range(n_images):
        plt.subplot(2, n_images, 1 + image_index)
        plot_image(images[image_index])
        plt.subplot(2, n_images, 1 + n_images + image_index)
        plot_image(reconstructions[image_index])

show_reconstructions(stacked_autoencoder)

In [ ]:
# Code17.10
# visualization 2d result

from sklearn.manifold import TSNE

X_valid_compressed = stacked_encoder.predict(X_valid)
tsne = TSNE()
X_valid_2D = tsne.fit_transform(X_valid_compressed)
X_valid_2D = (X_valid_2D - X_valid_2D.min()) / (X_valid_2D.max() - X_valid_2D.min())

plt.scatter(X_valid_2D[:, 0], X_valid_2D[:, 1], c = y_valid, s = 10, cmap = 'tab10')
plt.axis('off')
plt.show()

In [ ]:
# Code17.11
# Tinying weights model

class DenseTranspose(keras.layers.Layer):
    def __init__(self, dense, activation = None, **kwargs):
        self.dense = dense
        self.activation = keras.activations.get(activation)
        super().__init__(**kwargs)
    
    def build(self, batch_input_shape):
        self.biases = self.add_weight(name = 'bias', shape = [self.dense.input_shape[-1]], initializer = 'zeros')
        super().build(batch_input_shape)
    
    def call(self, inputs):
        z = tf.matmul(inputs, self.dense.weights[0], transpose_b = True)

        return self.activation(z + self.biases)

dense_1 = keras.layers.Dense(100, activation = 'selu')
dense_2 = keras.layers.Dense(30, activation = 'selu')

tied_encoder = keras.models.Sequential([
    keras.layers.Flatten(input_shape = [28, 28]),
    dense_1,
    dense_2
])

tied_decoder = keras.models.Sequential([
    DenseTranspose(dense_2, activation = 'selu'),
    DenseTranspose(dense_1, activation = 'sigmoid'),
    keras.layers.Reshape([28, 28])
])

tied_autoencoder = keras.models.Sequential([tied_encoder, tied_decoder])


In [ ]:
# Code17.12
# tied model train

tied_autoencoder.compile(loss = 'binary_crossentropy', optimizer = keras.optimizers.SGD(lr = 1.5), metrics = [rounded_accuracy])
history = tied_autoencoder.fit(X_train, X_train, epochs = 10, validation_data = (X_valid, X_valid))

In [ ]:
# Code17.13
# draw result

show_reconstructions(tied_autoencoder)
plt.show()

In [ ]:
# Code17.14
# 一次构建一个单层自动编码器，然后再将这些堆叠

def train_autoencoder(n_neurons, X_train, X_valid, loss, optimizer, n_epochs = 10, output_activation = None, metrics = None):
    n_inputs = X_train.shape[-1]
    
    encoder = keras.models.Sequential([
        keras.layers.Dense(n_neurons, activation = 'selu', input_shape = [n_inputs])              
    ])

    decoder = keras.models.Sequential([
        keras.layers.Dense(n_inputs, activation = output_activation)
    ])

    autoencoder = keras.models.Sequential([encoder, decoder])
    autoencoder.compile(optimizer, loss, metrics = metrics)
    autoencoder.fit(X_train, X_train, epochs = n_epochs, validation_data = (X_valid, X_valid))

    return encoder, decoder, encoder(X_train), encoder(X_valid)

In [ ]:
# Code17.15
# train

K = keras.backend
X_train_flat = K.batch_flatten(X_train)
X_valid_flat = K.batch_flatten(X_valid)

# 训练第一个
enc1, dec1, X_train_enc1, X_valid_enc1 = train_autoencoder(100, X_train_flat, X_valid_flat, 'binary_crossentropy', keras.optimizers.SGD(lr = 1.5), output_activation = 'sigmoid', metrics = [rounded_accuracy])
# 训练第二个
enc2, dec2, _, _ = train_autoencoder(30, X_train_enc1, X_valid_enc1, 'mse', keras.optimizers.SGD(lr = 0.05), output_activation = 'selu')
# 堆叠这两个
stacked_autoencoder_1_by_1 = keras.models.Sequential([
    keras.layers.Flatten(input_shape = [28, 28]),
    enc1,
    enc2,
    dec2,
    dec1,
    keras.layers.Reshape([28, 28])
])

In [ ]:
# Code17.16
# 画一下
show_reconstructions(stacked_autoencoder_1_by_1)
plt.show()

In [ ]:
# Code17.17
# 将最终模型再训练一下

stacked_autoencoder_1_by_1.compile(loss = 'binary_crossentropy', optimizer = keras.optimizers.SGD(lr = 0.1), metrics = [rounded_accuracy])
history = stacked_autoencoder_1_by_1.fit(X_train, X_train, epochs = 10, validation_data = (X_valid, X_valid))

In [ ]:
# Code17.18
# 再画一下

show_reconstructions(stacked_autoencoder_1_by_1)
plt.show()

In [ ]:
# Code17.19
# an simple Conv-autoencoder for mnist

conv_encoder = keras.models.Sequential([
    keras.layers.Reshape([28, 28, 1], input_shape = [28, 28]),
    keras.layers.Conv2D(16, kernel_size = 3, padding = 'SAME', activation = 'selu'),
    keras.layers.MaxPool2D(pool_size = 2),
    keras.layers.Conv2D(32, kernel_size = 3, padding = 'SAME', activation = 'selu'),
    keras.layers.MaxPool2D(pool_size = 2),
    keras.layers.Conv2D(64, kernel_size = 3, padding = 'SAME', activation = 'selu'),
    keras.layers.MaxPool2D(pool_size = 2)
])

conv_decoder = keras.models.Sequential([
    keras.layers.Conv2DTranspose(32, kernel_size = 3, strides = 2, padding = 'VALID', activation = 'selu', input_shape = [3, 3, 64]),
    keras.layers.Conv2DTranspose(16, kernel_size = 3, strides = 2, padding = 'SAME', activation = 'selu'),
    keras.layers.Conv2DTranspose(1, kernel_size = 3, strides = 2, padding = 'SAME', activation = 'sigmoid'),
    keras.layers.Reshape([28, 28])
])

conv_autoencoder = keras.models.Sequential([conv_encoder, conv_decoder])
conv_autoencoder.compile(loss = 'binary_crossentropy', optimizer = keras.optimizers.SGD(lr = 1.0), metrics = [rounded_accuracy])
history = conv_autoencoder.fit(X_train, X_train, epochs = 5, validation_data = (X_valid, X_valid))


In [ ]:
# Code17.20
# browser summary

print(conv_encoder.summary())
print(conv_decoder.summary())

In [ ]:
# Code17.21
# 画一下

show_reconstructions(conv_autoencoder)
plt.show()

In [ ]:
# Code17.22
# 循环自动编码器

recurrent_encoder = keras.models.Sequential([
    keras.layers.LSTM(100, return_sequences = True, input_shape = [28, 28]),
    keras.layers.LSTM(30)
])

recurrent_decoder = keras.models.Sequential([
    keras.layers.RepeatVector(28, input_shape = [30]),
    keras.layers.LSTM(100, return_sequences = True),
    keras.layers.TimeDistributed(keras.layers.Dense(28, activation = 'sigmoid'))
])

recurrent_autoencoder = keras.models.Sequential([recurrent_encoder, recurrent_decoder])
recurrent_autoencoder.compile(loss = 'binary_crossentropy', optimizer = keras.optimizers.SGD(lr = 0.1), metrics = [rounded_accuracy])
history = recurrent_autoencoder.fit(X_train, X_train, epochs = 10, validation_data = (X_valid, X_valid))


In [ ]:
# Code17.23
# 画一下

show_reconstructions(recurrent_autoencoder)
plt.show()

In [ ]:
# Code17.24
# denoise autoencoder

denoising_encoder = keras.models.Sequential([
    keras.layers.Flatten(input_shape = [28, 28]),
    keras.layers.GaussianNoise(0.2),
    keras.layers.Dense(100, activation = 'selu'),
    keras.layers.Dense(30, activation = 'selu')
])

denoising_decoder = keras.models.Sequential([
    keras.layers.Dense(100, activation = 'selu', input_shape = [30]),
    keras.layers.Dense(28 * 28, activation = 'sigmoid'),
    keras.layers.Reshape([28, 28])
])

denoising_autoencoder = keras.models.Sequential([denoising_encoder, denoising_decoder])
denoising_autoencoder.compile(loss = 'binary_crossentropy', optimizer = keras.optimizers.SGD(lr = 1.0), metrics = [rounded_accuracy])
history = denoising_autoencoder.fit(X_train, X_train, epochs = 10, validation_data = (X_valid, X_valid))


In [ ]:
# Code17.25
# 画一下

noise = keras.layers.GaussianNoise(0.2)
show_reconstructions(denoising_autoencoder, noise(X_valid, training = True))
plt.show()

In [ ]:
# Code17.26
# use dropout

dropout_encoder = keras.models.Sequential([
    keras.layers.Flatten(input_shape = [28, 28]),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(100, activation = 'selu'),
    keras.layers.Dense(30, activation = 'selu')
])

dropout_decoder = keras.models.Sequential([
    keras.layers.Dense(100, activation = 'selu', input_shape = [30]),
    keras.layers.Dense(28 * 28, activation = 'sigmoid'),
    keras.layers.Reshape([28, 28])
])

dropout_autoencoder = keras.models.Sequential([dropout_encoder, dropout_decoder])
dropout_autoencoder.compile(loss = 'binary_crossentropy', optimizer = keras.optimizers.SGD(lr = 1.0), metrics = [rounded_accuracy])
history = dropout_autoencoder.fit(X_train, X_train, epochs = 10, validation_data = (X_valid, X_valid))

In [ ]:
# Code17.27
# draw it

dropout = keras.layers.Dropout(0.5)
show_reconstructions(dropout_autoencoder, dropout(X_valid, training = True))

In [ ]:
# Code17.28
# Sparse autoencoder
# an simple example

simple_encoder = keras.models.Sequential([
    keras.layers.Flatten(input_shape = [28, 28]),
    keras.layers.Dense(100, activation = 'selu'),
    keras.layers.Dense(30, activation = 'sigmoid')
])

simple_decoder = keras.models.Sequential([
    keras.layers.Dense(100, activation = 'selu', input_shape = [30]),
    keras.layers.Dense(28 * 28, activation = 'sigmoid'),
    keras.layers.Reshape([28, 28])
])

simple_autoencoder = keras.models.Sequential([simple_encoder, simple_decoder])
simple_autoencoder.compile(loss = 'binary_crossentropy', optimizer = keras.optimizers.SGD(lr = 1.0), metrics = [rounded_accuracy])
history = simple_autoencoder.fit(X_train, X_train, epochs = 10, validation_data = (X_valid, X_valid))

In [ ]:
# Code17.29
# draw it

show_reconstructions(simple_autoencoder)
plt.show()

In [ ]:
# Code17.30
# use l1 regularization

sparse_l1_encoder = keras.models.Sequential([
    keras.layers.Flatten(input_shape = [28, 28]),
    keras.layers.Dense(100, activation = 'selu'),
    keras.layers.Dense(300, activation = 'sigmoid'),
    keras.layers.ActivityRegularization(l1 = 1e-3)
])

sparse_l1_decoder = keras.models.Sequential([
    keras.layers.Dense(100, activation = 'selu', input_shape = [300]),
    keras.layers.Dense(28 * 28, activation = 'sigmoid'),
    keras.layers.Reshape([28, 28])
])

sparse_l1_autoencoder = keras.models.Sequential([sparse_l1_encoder, sparse_l1_decoder])
sparse_l1_autoencoder.compile(loss = 'binary_crossentropy', optimizer = keras.optimizers.SGD(lr = 1.0), metrics = [rounded_accuracy])
history = sparse_l1_autoencoder.fit(X_train, X_train, epochs = 10, validation_data = (X_valid, X_valid))

In [ ]:
# Code17.31
# draw it

show_reconstructions(sparse_l1_autoencoder)
plt.show()

In [ ]:
# Code17.32
# KL autoencoder

K = keras.backend
kl_divergence = keras.losses.kullback_leibler_divergence

class KLDivergenceRegularizer(keras.regularizers.Regularizer):
    def __init__(self, weight, target = 0.1):
        self.weight = weight
        self.target = target
    def __call__(self, inputs):
        mean_activities = K.mean(inputs, axis = 0)
        
        return self.weight * (kl_divergence(self.target, mean_activities) + kl_divergence(1. - self.target, 1. - mean_activities))

kld_reg = KLDivergenceRegularizer(weight = 0.05, target = 0.1)
sparse_kl_encoder = keras.models.Sequential([
    keras.layers.Flatten(input_shape = [28, 28]),
    keras.layers.Dense(100, activation = 'selu'),
    keras.layers.Dense(300, activation = 'sigmoid', activity_regularizer = kld_reg)
])
sparse_kl_decoder = keras.models.Sequential([
    keras.layers.Dense(100, activation = 'selu', input_shape = [300]),
    keras.layers.Dense(28 * 28, activation = 'sigmoid'),
    keras.layers.Reshape([28, 28]) 
])

sparse_kl_autoencoder = keras.models.Sequential([sparse_kl_encoder, sparse_kl_decoder])
sparse_kl_autoencoder.compile(loss = 'binary_crossentropy', optimizer = keras.optimizers.SGD(lr = 1.0), metrics = [rounded_accuracy])
history = sparse_kl_autoencoder.fit(X_train, X_train, epochs = 10, validation_data = (X_valid, X_valid))

In [ ]:
 # Code17.33
 # draw it

 show_reconstructions(sparse_kl_autoencoder)
 plt.show()

In [ ]:
# Code17.34
# variational autoencoder

class Sampling(keras.layers.Layer):
    def call(self, inputs):
        mean, log_var = inputs
        return K.random_normal(tf.shape(log_var)) * K.exp(log_var / 2) + mean

codings_size = 10

# encoder
inputs = keras.layers.Input(shape = [28, 28])
z = keras.layers.Flatten()(inputs)
z = keras.layers.Dense(150, activation = 'selu')(z)
z = keras.layers.Dense(100, activation = 'selu')(z)
codings_mean = keras.layers.Dense(codings_size)(z)
codings_log_var = keras.layers.Dense(codings_size)(z)
codings = Sampling()([codings_mean, codings_log_var])
variational_encoder = keras.models.Model(inputs = [inputs], outputs = [codings_mean, codings_log_var, codings])

# decoder
decoder_inputs = keras.layers.Input(shape = [codings_size])
x = keras.layers.Dense(100, activation = 'selu')(decoder_inputs)
x = keras.layers.Dense(150, activation = 'selu')(x)
x = keras.layers.Dense(28 * 28, activation = 'sigmoid')(x)
outputs = keras.layers.Reshape([28, 28])(x)
variational_decoder = keras.models.Model(inputs = [decoder_inputs], outputs = [outputs])

_, _, codings = variational_encoder(inputs)
reconstructions = variational_decoder(codings)
variational_autoencoder = keras.models.Model(inputs = [inputs], outputs = [reconstructions])

latent_loss = -0.5 * K.sum(1 + codings_log_var - K.exp(codings_log_var) - K.square(codings_mean), axis = -1)
variational_autoencoder.add_loss(K.mean(latent_loss) / 784.0)
variational_autoencoder.compile(loss = 'binary_crossentropy', optimizer = 'rmsprop', metrics = [rounded_accuracy])
history = variational_autoencoder.fit(X_train, X_train, epochs = 25, batch_size = 128, validation_data = (X_valid, X_valid))

In [ ]:
# Code17.35
# draw it

show_reconstructions(variational_autoencoder)
plt.show()

In [ ]:
# Code17.36
# draw images func

def plot_mutiple_images(images, n_cols = None):
    n_cols = n_cols or len(images)
    n_rows = (len(images) - 1) // n_cols + 1
    if images.shape[-1] == 1:
        images = np.squeeze(images, axis = -1)
    plt.figure(figsize = (n_cols, n_rows))
    for index, image in enumerate(images):
        plt.subplot(n_rows, n_cols, index + 1)
        plt.imshow(image, cmap = 'binary')
        plt.axis('off')

In [ ]:
# Code17.37
# generate some image

codings = tf.random.normal(shape = [12, codings_size])
images = variational_decoder(codings).numpy()
plot_mutiple_images(images, 4)

In [ ]:
# Code17.38
#

codings_grid = tf.reshape(codings, [1, 3, 4, codings_size])
larger_grid = tf.image.resize(codings_grid, size = [5, 7])
interpolated_codings = tf.reshape(larger_grid, [-1, codings_size])
images = variational_decoder(interpolated_codings).numpy()

plt.figure(figsize = (7, 5))
for index, image in  enumerate(images):
    plt.subplot(5, 7, index + 1)
    if index % 7 % 2 == 0 and index // 7 % 2 == 0:
        plt.gca().get_xaxis().set_visible(False)
        plt.gca().get_yaxis().set_visible(False)
    else:
        plt.axis('off')
    plt.imshow(image, cmap = 'binary')

plt.show()

In [ ]:
# Code17.39
# gan

# 编码数量
codings_size = 30

#  生成器
generator = keras.models.Sequential([
    keras.layers.Dense(100, activation = 'selu', input_shape = [codings_size]),
    keras.layers.Dense(150, activation = 'selu'),
    keras.layers.Dense(28 * 28, activation = 'sigmoid'),
    keras.layers.Reshape([28, 28])
])

# 判别器
discriminator = keras.models.Sequential([
    keras.layers.Flatten(input_shape = [28, 28]),
    keras.layers.Dense(150, activation = 'selu'),
    keras.layers.Dense(100, activation = 'selu'),
    keras.layers.Dense(1, activation = 'sigmoid')
])

gan = keras.models.Sequential([generator, discriminator])

discriminator.compile(loss = 'binary_crossentropy', optimizer = 'rmsprop')
discriminator.trainable = False
gan.compile(loss = 'binary_crossentropy', optimizer = 'rmsprop')

In [ ]:
# Code17.40
# customalized train func

def train_gan(gan, dataset, batch_size, codings_size, n_epochs = 50):
    generator, discriminator = gan.layers
    for epoch in range(n_epochs):
        print('Epoch{}/{}'.format(epoch + 1, n_epochs))
        for X_batch in dataset:
            # 训练判别器
            noise = tf.random.normal(shape = [batch_size, codings_size])
            generated_images = generator(noise)
            X_fake_and_real = tf.concat([generated_images, X_batch], axis = 0)
            y1 = tf.constant([[0.]] * batch_size + [[1.]] * batch_size)
            discriminator.trainable = True
            discriminator.train_on_batch(X_fake_and_real, y1)

            # 训练生成器
            noise = tf.random.normal(shape = [batch_size, codings_size])
            y2 = tf.constant([[1.]] * batch_size)
            discriminator.trainable = False
            gan.train_on_batch(noise, y2)
        
        plot_mutiple_images(generated_images, 8)
        plt.show()

In [ ]:
# Code17.41
# create train set

batch_size = 32
dataset = tf.data.Dataset.from_tensor_slices(X_train).shuffle(1000)
dataset = dataset.batch(batch_size, drop_remainder = True).prefetch(1)

In [ ]:
# Code17.42
# train 1 epochs

train_gan(gan, dataset, batch_size, codings_size, n_epochs = 1)

In [ ]:
# Code17.43
# generate some picture

noise = tf.random.normal(shape = [batch_size, codings_size])
generated_images = generator(noise)
plot_mutiple_images(generated_images, 8)

In [ ]:
# Code17.44
# train 50 epochs

train_gan(gan, dataset, batch_size, codings_size)

In [ ]:
# Code17.45
# DCGAN

codings_size = 100

generator = keras.models.Sequential([
    keras.layers.Dense(7 * 7 * 128, input_shape = [codings_size]),
    keras.layers.Reshape([7, 7, 128]),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2DTranspose(64, kernel_size = 5, strides = 2, padding = 'SAME', activation = 'selu'),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2DTranspose(1, kernel_size = 5, strides = 2, padding = 'SAME', activation = 'tanh')
])

discriminator = keras.models.Sequential([
    keras.layers.Conv2D(64, kernel_size = 5, strides = 2, padding = 'SAME', activation = keras.layers.LeakyReLU(0.2), input_shape = [28, 28, 1]),
    keras.layers.Dropout(0.4),
    keras.layers.Conv2D(128, kernel_size = 5, strides = 2, padding = 'SAME', activation = keras.layers.LeakyReLU(0.2)),
    keras.layers.Dropout(0.4),
    keras.layers.Flatten(),
    keras.layers.Dense(1, activation = 'sigmoid')
])

gan = keras.models.Sequential([generator, discriminator])
discriminator.compile(loss = 'binary_crossentropy', optimizer = 'rmsprop')
discriminator.trainable = False
gan.compile(loss = 'binary_crossentropy', optimizer = 'rmsprop')

In [ ]:
# Code17.46
#  create dataset

X_train_dcgan = X_train.reshape(-1, 28, 28, 1) * 2. - 1.

batch_size = 32
dataset = tf.data.Dataset.from_tensor_slices(X_train_dcgan)
dataset = dataset.shuffle(1000)
dataset = dataset.batch(batch_size, drop_remainder = True).prefetch(1)


In [ ]:
# Code17.47
# train 

train_gan(gan, dataset, batch_size, codings_size)

In [ ]:
# Code17.48
# generate some images

noise = tf.random.normal(shape = [batch_size,codings_size])
generated_images = generator(noise)
plot_mutiple_images(generated_images, 8)
plt.show()